# Aprendizagem Automática Avançada

**Alunos:**
    João Raimundo (57454), Rui Roque (57588)

## Assigment #4 - Hidden Markov Models

## Problema 2

### 0.1 - Importação das Bibliotecas

In [269]:
import numpy as np
import math
import pandas as pd
from hmmlearn.hmm import MultinomialHMM
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error

### 0.2 - Definição de Funções

In [270]:
def loadData(data, rolls = True):
    arr = []
    for i in data:
        arr = np.append(arr, i)

    arr = np.array([arr]).T
    
    if(rolls == True):
        arr = arr.astype("int")

    return arr

### 0.3 - Definição dos Dados

O conjunto de dados **X_train** foi denifido com a amostra "Rolls" do enunciado sem os 10 últimos caracteres/records.

In [271]:
# X_train
X_train = loadData("66415321616211523465321435663426165523423231514246")

X_train

array([[6],
       [6],
       [4],
       [1],
       [5],
       [3],
       [2],
       [1],
       [6],
       [1],
       [6],
       [2],
       [1],
       [1],
       [5],
       [2],
       [3],
       [4],
       [6],
       [5],
       [3],
       [2],
       [1],
       [4],
       [3],
       [5],
       [6],
       [6],
       [3],
       [4],
       [2],
       [6],
       [1],
       [6],
       [5],
       [5],
       [2],
       [3],
       [4],
       [2],
       [3],
       [2],
       [3],
       [1],
       [5],
       [1],
       [4],
       [2],
       [4],
       [6]])

Apesar de não ser usado, o conjunto de dados **y_train** (target) foi definido pelos com a amostra "Dice" do enunciado sem os 10 últimos caracteres/records.

In [272]:
# y_train
y_train = loadData("LLLLLLLLLLLLLLFFFFFFLLLLLLLLLLLLLLFFFFFFFFFFFFFFFF", rolls=False)

y_train

array([['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F'],
       ['F']], dtype='<U32')

O conjunto de dados **X_test** foi denifido pelos últimos 10 caracteres/records da amostra "Rolls" do enunciado.

Uma vez que o modelo "MultinomialHMM" da biblioteca "hmmlearn.hmm" está programado de forma que assume como index inicial dos arrays o valor 1, em vez do index zero. Pelo que subtraimos -1 a cada um dos valores definidos no array (X_test), caso contrario o modelo quando testado não reconheceria os valores treinados pelo conjunto de treino.

In [273]:
# X_test
X_test = loadData("4156663246")

X_test = X_test-1
X_test

array([[3],
       [0],
       [4],
       [5],
       [5],
       [5],
       [2],
       [1],
       [3],
       [5]])

O conjunto de dados **y_test** foi denifido pelos últimos 10 caracteres/records da amostra "Dice" do enunciado.


In [274]:
#y_test
y_test = loadData("FFLLLLLLLL", rolls=False)

y_test

array([['F'],
       ['F'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L'],
       ['L']], dtype='<U32')

Adicionalmente foi criada uma variável y_test_bin onde os valores do conjunto y_test foram mapeados para um formato binário: F=0, L=1.

In [275]:
# mapeamento dos valores da variável y_test para um formato binário: F=0, L=1
y_test_bin = []

for i in "FFLLLLLLLL":
    if(i == "F"):
        y_test_bin = np.append(y_test_bin, 0)
    else:
        y_test_bin = np.append(y_test_bin, 1)

y_test_bin = y_test_bin.astype(int)

y_test_bin

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

### 1. Construção, Treino do Modelo Classificatório MultinomialHMM e Previsão dos Valores do Conjunto de Teste

#### 1.1. Alínea A)

* Definição dos states para os dois dados: Fair & Loaded (π1, π2 respetivamente)
* Criação das *stationary probabilities* para os dois states
* Definição dos valores a serem usados na matrix de transição 

* Criação do Modelo Classificatório: MultinomialHMM() com os dois states criados
* Definição do estado inicial das stationary probabilities no modelo MHMM com os valores de π
* Definição do estado da  matriz de transição no modelo classificatório MHMM com as probabilidades de transição entre estados
* Definição do estado da matriz de emissão no modelo classificatório MHMM com as probabilidades de calhar as faces em cada um dos dados
* Previsão dos valores do conjunto X_test
* Cálculo dos valores de log likelihood (log probability) para o conjunto X_test
* (Adicional) Cálculo dos erros - Comparação entre 1-accuracy e mean squared error
* (Adicional) Cálculo das várias métricas de score (ClassificationReport)

In [276]:
# definição dos states: dados = Fair & Loaded
states = ["F", "L"] 

# stationary probabilities
pi1 = 0.1/(0.05+0.1) # fair
pi2 = 0.05/(0.05+0.1) # load

# probabilidade de o estado fair passar para o estado loaded
alpha = 0.05
# probabilidade de o estado loaded passar para o estado fair
beta = 0.1
# probabilidade de o estado fair passar para o estado fair
aux1 = 1-alpha
# probabilidade de o estado loaded passar para o estado loaded
aux2 = 1-beta

# Criação do modelo MHMM
model = MultinomialHMM(n_components=2)

# Definição do estado inicial das stationary probabilities no modelo MHMM 
model.startprob_= [pi1,pi2]

# Definição do estado da matrix de transição no modelo MHMM 
model.transmat_=  np.array([[aux1, alpha],
                [beta, aux2]])
                

# emission matrix - probabilidade de sair cada face do dado em cada estado
model.emissionprob_ = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],[1/10, 1/10, 1/10, 1/10, 1/10, 5/10]])

# predictions 
preds = model.predict(X_test)

# log probability
score = model.score(X_test)

print("preds {0:F, L:1}:", preds)
print("rolls (X_test): 4, 1, 5, 6, 6, 6, 3, 2, 4, 6") # X_test
print ("dice (preds):  ", ", ".join(map(lambda x: states[x], preds)), "\n\n")

print("Log Likelihood: ", score, "\n")

# cálculo do erro e metricas de score
print("error: ", 1 - accuracy_score(y_test_bin, preds))
print("mean squared error: ", mean_squared_error(y_test_bin, preds), "\n")
print("Classification Report:\n\n", classification_report(y_test_bin, preds, zero_division=False))

preds {0:F, L:1}: [1 1 1 1 1 1 1 1 1 1]
rolls (X_test): 4, 1, 5, 6, 6, 6, 3, 2, 4, 6
dice (preds):   L, L, L, L, L, L, L, L, L, L 


Log Likelihood:  -16.98461390418157 

error:  0.19999999999999996
mean squared error:  0.2 

Classification Report:

               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.80      1.00      0.89         8

    accuracy                           0.80        10
   macro avg       0.40      0.50      0.44        10
weighted avg       0.64      0.80      0.71        10



O likelihood mede quão bem o modelo se ajusta aos dados.

#### 1.2. Alínea B)

Na alínea B procedemos com a mesma metodologia da alínea A), porém não foram definidos os estados iniciais das "stationary probabilities" e da matriz de transição no modelo MHMM.

In [277]:
# Criação do modelo MHMM
model = MultinomialHMM(n_components=2)

# emission matrix - probabilidade de sair cada face do dado em cada estado
model.emissionprob_ = np.array([[1/6, 1/6, 1/6, 1/6, 1/6, 1/6],[1/10, 1/10, 1/10, 1/10, 1/10, 5/10]])

# fit do modelo com os o set X_train
model.fit(X_train)

# predictions 
preds = model.predict(X_test)

# log Likelihood
score = model.score(X_test)


print("preds {0:F, L:1}:", preds)
print("rolls (X_test): 4, 1, 5, 6, 6, 6, 3, 2, 4, 6") # X_test
print ("dice (preds):  ", ", ".join(map(lambda x: states[x], preds)), "\n\n")

print("Log Likelihood: ", score, "\n")

# cálculo do erro e metricas de score
print("error: ", 1 - accuracy_score(y_test_bin, preds))
print("mean squared error: ", mean_squared_error(y_test_bin, preds), "\n")
print("Classification Report:\n\n", classification_report(y_test_bin, preds, zero_division=False))

preds {0:F, L:1}: [1 1 1 1 1 1 1 1 1 0]
rolls (X_test): 4, 1, 5, 6, 6, 6, 3, 2, 4, 6
dice (preds):   L, L, L, L, L, L, L, L, L, F 


Log Likelihood:  -inf 

error:  0.30000000000000004
mean squared error:  0.3 

Classification Report:

               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.78      0.88      0.82         8

    accuracy                           0.70        10
   macro avg       0.39      0.44      0.41        10
weighted avg       0.62      0.70      0.66        10



Quando não definidos os estados iniciais das "stationary probabilities" e da matriz de transição verificamos que o valor da log-likelihood é - infinito.

Quanto mais próximo de zero for o valor da log-likelihood melhor será o ajusto do modelo aos dados. Desta forma uma vez que o modelo da alínea A) obteve o valor de log-likelihood= ~ -17 e na alínea B) obtivemos um log-likelihood= -infinito verifica-se que o modelo treinado na alínea A) consegue-se ajustar melhor aos dados que o modelo da alínea B).

#### 1.3. Alínea C)

Na alínea C) iremos comparar os valores de log-likelihood para um modelo quando treinado com um conjunto de dados de 100 e 1000 amostras, e testado posteriormente com um y_test constituído por 30 amostras para:

* Um modelo onde não é definido o estado inicial das "stationary probabilities", matriz de transmissão e emissão (1.3.1.)
* Um modelo onde todos os parâmetros são desconhecidos incluindo o número de estados, testando valores para o número de estados, [1-4] (1.3.2.)

### 1.3.0.1. Importação dos dados

À semelhança das alíneas anteriores subtraimos também -1 a todos os valores do array dos rolls.

In [278]:
# Rolls (features)
rolls = pd.read_csv("rolls.txt", sep="\t", header=None)

rolls = np.array(rolls)

rolls = rolls - 1

print("Rolls dataset:",rolls, "\n")
print("Rolls dataset lenght:",len(rolls),"\n\n")

# Dice (target)
dice = pd.read_csv("dice.txt", sep="\t", header=None)

dice = np.array(dice)

print("Dice dataset:",dice, "\n")
print("Dice dataset lenght:",len(dice),"\n\n")

Rolls dataset: [[0]
 [4]
 [2]
 ...
 [2]
 [4]
 [3]] 

Rolls dataset lenght: 1100 


Dice dataset: [['F']
 ['F']
 ['F']
 ...
 ['F']
 ['F']
 ['F']] 

Dice dataset lenght: 1100 




### 1.3.0.2. Definição dos conjuntos de dados

In [279]:
## Test set features

# seleção dos 100 primeiros valores no array
x_train_rolls100 = rolls[0:100]

# seleção dos 1000 primeiros valores no array
x_train_rolls1000 = rolls[0:1000]

# seleção dos últimos 30 valores da amostra do array para definir o X_test
x_test = rolls[1070:1100] # 30 amostras

print("x_train_rolls100:", len(x_train_rolls100), "\n")
print("x_train_rolls1000:", len(x_train_rolls1000), "\n")
print("x_test:", len(x_test))

x_train_rolls100: 100 

x_train_rolls1000: 1000 

x_test: 30


In [280]:
# definição de uma função para mapeamento dos valores da variável y_test para um formato binário: F=0, L=1
def convertToBin(arr):
    y_test_bin = []

    for i in arr:
        if(i[0] == "F"):
            y_test_bin = np.append(y_test_bin, 0)
        else:
            y_test_bin = np.append(y_test_bin, 1)

    y_test_bin = y_test_bin.astype(int)
    return y_test_bin

In [281]:
# seleção dos 100 primeiros valores no array - train set target
y_train_dice100 = dice[0:100]

# seleção dos 1000 primeiros valores no array - train set target
y_train_dice1000 = dice[0:1000]

# seleção dos últimos 30 valores da amostra do array para definir o y_test
y_test = dice[1001:1031]

# conversão dos valores da amostra para o formato binário
y_test_bin = convertToBin(y_test)

### 1.3.1. (I) Rolls com 100 amostras e "stationary probabilities", matriz de transição e emissão desconhecidas

#### 1.3.1.1. Construção, Treino do Modelo Classificatório MultinomialHMM e Previsão dos Valores do Conjunto de Teste

In [282]:
model = MultinomialHMM(n_components=2, random_state=42)

model.fit(x_train_rolls100)

# predictions 
preds100 = model.predict(x_test)

# log Likelihood
score = model.score(x_test)

print("preds {0:F, L:1}:", preds100)
print ("dice (preds):  ", ", ".join(map(lambda x: states[x], preds100)), "\n\n")

print("Log Likelihood: ", score, "\n\n")

# cálculo do erro e metricas de score
print("error: ", 1 - accuracy_score(y_test_bin, preds100))
print("mean squared error: ", mean_squared_error(y_test_bin, preds100), "\n")
print("Classification Report:\n\n", classification_report(y_test_bin, preds100, zero_division=False))

preds {0:F, L:1}: [0 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1]
dice (preds):   F, L, L, L, F, L, L, L, L, L, L, F, L, F, F, L, L, L, L, F, L, L, F, F, F, F, F, F, F, L 


Log Likelihood:  -58.423164121333 


error:  0.6333333333333333
mean squared error:  0.6333333333333333 

Classification Report:

               precision    recall  f1-score   support

           0       0.85      0.39      0.54        28
           1       0.00      0.00      0.00         2

    accuracy                           0.37        30
   macro avg       0.42      0.20      0.27        30
weighted avg       0.79      0.37      0.50        30



### 1.3.2. (I) Rolls com 1000 amostras e "stationary probabilities", matriz de transição e emissão desconhecidas

#### 1.3.1.1. Construção, Treino do Modelo Classificatório MultinomialHMM e Previsão dos Valores do Conjunto de Teste

In [283]:
model = MultinomialHMM(n_components=2, random_state=42)

model.fit(x_train_rolls1000)

# predictions 
preds1000 = model.predict(x_test)

# log Likelihood
score = model.score(x_test)

print("preds {0:F, L:1}:", preds1000)
print ("dice (preds):  ", ", ".join(map(lambda x: states[x], preds1000)), "\n\n")

print("Log Likelihood: ", score, "\n\n")


# cálculo do erro e metricas de score
print("error: ", 1 - accuracy_score(y_test_bin, preds1000))
print("mean squared error: ", mean_squared_error(y_test_bin, preds1000), "\n")
print("Classification Report:\n\n", classification_report(y_test_bin, preds1000, zero_division=False))

preds {0:F, L:1}: [0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
dice (preds):   F, F, L, L, F, F, F, F, L, L, L, F, L, F, F, F, F, F, F, F, F, L, F, F, F, F, F, F, F, F 


Log Likelihood:  -55.79630141304058 


error:  0.30000000000000004
mean squared error:  0.3 

Classification Report:

               precision    recall  f1-score   support

           0       0.91      0.75      0.82        28
           1       0.00      0.00      0.00         2

    accuracy                           0.70        30
   macro avg       0.46      0.38      0.41        30
weighted avg       0.85      0.70      0.77        30



Verifica-se com os resultados obtidos nos pontos 1.3.1. e 1.3.2. que o modelo treinado com 1000 amostras do conjunto Rolls, obteve um valor de log-likelihood (-55.8) superior ao do modelo treinado com 100 amostras (-58.4). Este resultado poderá indicar que quanto maior for a dimensão do conjunto de dados melhor será o ajuste do modelo aos dados, devido a um treino melhor suportado.

Adicionalmente, esta conclusão pode ser complementada pelos valores dos erros obtidos em cada um dos modelos - o modelo com a amostra 1000 do conjunto Rolls obteve menos 33% de erro que o modelo com uma amostragem de 100 - bem como fazendo referência às várias métricas de score obtidas. A macro average f1-score do modelo com 1000 amostras (0.41) é superior à do modelo com 100 amostras (0.27).

### 1.3.3.0. Definição de Funções para treinar e testar o modelo

In [284]:
def classifyMHMM_model(n_components, x_train, x_test, y_test):
    #states = ["F", "L"] 
    model = MultinomialHMM(n_components=n_components, random_state=42)

    model.fit(x_train)

    # predictions 
    preds = model.predict(x_test)

    # log Likelihood
    score = model.score(x_test)

    print("preds {0:F, L:1}:", preds)
    print("\n\n")

    print("Log Likelihood: ", score, "\n\n")

    # cálculo do erro e metricas de score
    print("error: ", 1 - accuracy_score(y_test, preds))
    print("mean squared error: ", mean_squared_error(y_test, preds), "\n")
    print("Classification Report:\n\n", classification_report(y_test, preds, zero_division=False))
    print("----------------------------------------------------------\n")

### 1.3.3.1. (II) Rolls com 100 amostras com todos o parâmetro desconhecidos e alterando o valor dos estados [1-4]

In [285]:
# 100

i = 1
while(i < 5):
    print("Number of states: ", i)
       
    classifyMHMM_model(i, x_train_rolls100, x_test, y_test_bin)
    i += 1
    print("\n")

Number of states:  1
preds {0:F, L:1}: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



Log Likelihood:  -56.87454481610908 


error:  0.06666666666666665
mean squared error:  0.06666666666666667 

Classification Report:

               precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.00      0.00      0.00         2

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30

----------------------------------------------------------



Number of states:  2
preds {0:F, L:1}: [0 1 1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 0 0 0 0 0 0 1]



Log Likelihood:  -58.423164121333 


error:  0.6333333333333333
mean squared error:  0.6333333333333333 

Classification Report:

               precision    recall  f1-score   support

           0       0.85      0.39      0.54        28
           1       0.00     

### 1.3.3.2. (II) Rolls com 1000 amostras com todos o parâmetro desconhecidos e alterando o valor dos estados [1-4]

In [286]:
# 1000

i = 1
while(i < 5):
    print("Number of states: ", i)
       
    classifyMHMM_model(i, x_train_rolls1000, x_test, y_test_bin)
    i += 1
    print("\n")

Number of states:  1
preds {0:F, L:1}: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]



Log Likelihood:  -54.44137623655621 


error:  0.06666666666666665
mean squared error:  0.06666666666666667 

Classification Report:

               precision    recall  f1-score   support

           0       0.93      1.00      0.97        28
           1       0.00      0.00      0.00         2

    accuracy                           0.93        30
   macro avg       0.47      0.50      0.48        30
weighted avg       0.87      0.93      0.90        30

----------------------------------------------------------



Number of states:  2
preds {0:F, L:1}: [0 0 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]



Log Likelihood:  -55.79630141304058 


error:  0.30000000000000004
mean squared error:  0.3 

Classification Report:

               precision    recall  f1-score   support

           0       0.91      0.75      0.82        28
           1       0.00      0.00      0

Resultados de log-likelihood obtidos em ambos os modelos:

Rolls 100 amostras - target 30 amostras:

 * 1 state:  -56.87454481610908 
 * 2 states: -58.423164121333
 * 3 states: -57.94501734515244
 * 4 states: -58.090873485773145 
 
Rolls 1000 amostras - target 30 amostras:
* 1 state:  -54.44137623655621
* 2 states: -55.79630141304058
* 3 states: -54.371564322598914
* 4 states: -54.32993998155247

</br>

Verifica-se que treinados sem parâmetros, para o modelo com 100 amostras o maior valor de log-likelihood obtido foi para o modelo treinado com apenas 1 estado (-56.9), enquanto que para o modelo com 1000 amostras o maior log-likelihood obtido para para o modelo treinado com 4 estados. O erro obtido para o modelo com 100 amostras treinado com 1 estado tem o valor de ~ 7%, enquanto que no modelo de 1000 amostras treinado com 4 estados foi obtido um erro no valor de ~ 63%.

Deste modo, podemos concluir que para um modelo treinado sem parâmetros e apenas testando o número dos estados, talvez seja melhor o uso de um conjunto de dados mais pequeno, pois o erro obtido pelo modelo treinado com o conjunto de 1000 amostras (~ 63%) é muito superior comparativamente ao de 100 amostras (~ 7%). Apesar do valor de log-likelihood ser superior no modelo com 1000 amostras, mas poderá estar relacionado com o facto desta métrica estar correlacionada com a dimensão da amostra utilizada para treinar o modelo - quanto maior for o número de dados utilizados para treinar o modelo, melhor será o ajuste do modelo aos dados.